In [1]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#SQL DATABASE CREATION
# CREATE TABLE oscars(award VARCHAR,
# 					release_year INT,
# 					name VARCHAR,
# 					title VARCHAR,
# 					winner BOOLEAN
# 					);

In [3]:
#Import CSV File
csv_file = "the_oscar_award.csv"
Oscar_df = pd.read_csv(csv_file)
Oscar_df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [12]:
Oscar_group=Oscar_df.groupby('film')
OscarCount = Oscar_group.count()

In [25]:
Oscar_reset=OscarCount.reset_index()
Oscar_reset.head()

,film,year_film,year_ceremony,ceremony,category,name,winner
0,"$1,000 a Minute",1,1,1,1,1,1
1,'38',1,1,1,1,1,1
2,'Crocodile' Dundee,1,1,1,1,1,1
3,'Round Midnight,2,2,2,2,2,2
4,(A) Torzija [(A) Torsion],1,1,1,1,1,1


In [30]:
OscarID = Oscar_reset[['film','year_film']]
OscarID = OscarID.rename(columns={"year_film":"oscar_nominations"})
OscarID

,film,oscar_nominations
0,"$1,000 a Minute",1
1,'38',1
2,'Crocodile' Dundee,1
3,'Round Midnight,2
4,(A) Torzija [(A) Torsion],1
...,...,...
4828,the end,1
4829,tom thumb,1
4830,À Nous la Liberté,1
4831,Ådalen '31,1


In [17]:
#Clean Dataframe
Oscar1_df = Oscar_df[['year_film','category','name','film','winner']]
Oscar1_df = Oscar1_df.rename(columns={"category":"award",
                                          "year_film":"release_year",
                                          "name":"name",
                                          "film":"title",
                                          "winner":"winner"})
Oscar1_df = Oscar1_df.set_index("award")
Oscar1_df.head()

,release_year,name,title,winner
award,,,,
ACTOR,1927,Richard Barthelmess,The Noose,False
ACTOR,1927,Emil Jannings,The Last Command,True
ACTRESS,1927,Louise Dresser,A Ship Comes In,False
ACTRESS,1927,Janet Gaynor,7th Heaven,True
ACTRESS,1927,Gloria Swanson,Sadie Thompson,False


In [18]:
OscarNom_df = Oscar1_df.loc[["BEST PICTURE","DIRECTING","ACTOR","ACTRESS"]]
OscarNom_df

,release_year,name,title,winner
award,,,,
BEST PICTURE,1962,"Sam Spiegel, Producer",Lawrence of Arabia,True
BEST PICTURE,1962,"Darryl F. Zanuck, Producer",The Longest Day,False
BEST PICTURE,1962,"Morton Da Costa, Producer",Meredith Willson's The Music Man,False
BEST PICTURE,1962,"Aaron Rosenberg, Producer",Mutiny on the Bounty,False
BEST PICTURE,1962,"Alan J. Pakula, Producer",To Kill a Mockingbird,False
...,...,...,...,...
ACTRESS,1975,Isabelle Adjani,The Story of Adele H.,False
ACTRESS,1975,Ann-Margret,Tommy,False
ACTRESS,1975,Louise Fletcher,One Flew over the Cuckoo's Nest,True


In [19]:
OscarWin_df = OscarNom_df.loc[OscarNom_df["winner"] == True, :]
OscarWin_df.head(1)

,release_year,name,title,winner
award,,,,
BEST PICTURE,1962,"Sam Spiegel, Producer",Lawrence of Arabia,True


In [20]:
#Reset Index on OscarNom_df
OscarNom_df = OscarNom_df.reset_index()
OscarNom_df = OscarNom_df[['award','release_year','name','title','winner']]
OscarNom_df

,award,release_year,name,title,winner
0,BEST PICTURE,1962,"Sam Spiegel, Producer",Lawrence of Arabia,True
1,BEST PICTURE,1962,"Darryl F. Zanuck, Producer",The Longest Day,False
2,BEST PICTURE,1962,"Morton Da Costa, Producer",Meredith Willson's The Music Man,False
3,BEST PICTURE,1962,"Aaron Rosenberg, Producer",Mutiny on the Bounty,False
4,BEST PICTURE,1962,"Alan J. Pakula, Producer",To Kill a Mockingbird,False
...,...,...,...,...,...
1245,ACTRESS,1975,Isabelle Adjani,The Story of Adele H.,False
1246,ACTRESS,1975,Ann-Margret,Tommy,False
1247,ACTRESS,1975,Louise Fletcher,One Flew over the Cuckoo's Nest,True
1248,ACTRESS,1975,Glenda Jackson,Hedda,False


In [21]:
#Connect to local database
rds_connection_string = "postgres:postgres@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [22]:
#Verify
engine.table_names()

['oscars']

In [23]:
OscarNom_df.to_sql(name='oscars', con=engine, if_exists='append', index=False)

In [24]:
#Confirm successful upload
pd.read_sql_query('select * from oscars', con=engine)

,award,release_year,name,title,winner
0,BEST PICTURE,1962,"Sam Spiegel, Producer",Lawrence of Arabia,True
1,BEST PICTURE,1962,"Darryl F. Zanuck, Producer",The Longest Day,False
2,BEST PICTURE,1962,"Morton Da Costa, Producer",Meredith Willson's The Music Man,False
3,BEST PICTURE,1962,"Aaron Rosenberg, Producer",Mutiny on the Bounty,False
4,BEST PICTURE,1962,"Alan J. Pakula, Producer",To Kill a Mockingbird,False
...,...,...,...,...,...
2495,ACTRESS,1975,Isabelle Adjani,The Story of Adele H.,False
2496,ACTRESS,1975,Ann-Margret,Tommy,False
2497,ACTRESS,1975,Louise Fletcher,One Flew over the Cuckoo's Nest,True
2498,ACTRESS,1975,Glenda Jackson,Hedda,False


In [31]:
rds_connection_str = "postgres:postgres@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_str}')

In [34]:
#Verify
engine.table_names()

['oscars', 'oscarindex']

In [37]:
OscarID.to_sql(name='oscarindex', con=engine, if_exists='append', index=False)

In [38]:
#Confirm successful upload
pd.read_sql_query('select * from oscarindex', con=engine)

,id,film,oscar_nominations
0,1,"$1,000 a Minute",1
1,2,'38',1
2,3,'Crocodile' Dundee,1
3,4,'Round Midnight,2
4,5,(A) Torzija [(A) Torsion],1
...,...,...,...
4828,4829,the end,1
4829,4830,tom thumb,1
4830,4831,À Nous la Liberté,1
4831,4832,Ådalen '31,1
